In [16]:
import numpy as np
import tensorflow
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import math

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [19]:
MODEL_PATH = '/saved_models/nlp_models'

In [20]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###**Text preprocessing functions**

In [21]:
from __future__ import unicode_literals
from hazm import *
import re
import random
from string import punctuation

def text_preprocess(text):
  normalizer = Normalizer()
  text = normalizer.normalize(text)
  text = re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)
  return text

def text_scramble(text):
  words = text.split()
  random.shuffle(words)
  return ' '.join(words)


###**Loading and processing Corpus 1**
(VOA FARSI 2003-08)

In [22]:
path = 'data/voa_fa_2003-2008_orig.txt'
with open(path, 'r', encoding='utf-8') as f:
  text = ' '.join([line.strip() for line in f.readlines() if not line.startswith('#')])
  text = text.split('.')
  sents = random.sample(text, 40000)

In [23]:
MAX_LEN = 190
sents_cleaned = [text_preprocess(s) for s in sents]
sents_ready = filter(lambda s: len(s) < MAX_LEN, sents_cleaned)
sents_data = list(sents_ready)
print(len(sents_data))
random.shuffle(sents_data)
sents_data = sents_data[:35000]  # make dataset size 35000 for ease of computation

35498


In [24]:
tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<UNK>')
tokenizer.fit_on_texts(sents_data)
tokenizer.word_index['<PAD>'] = 0
last_idx = len(tokenizer.word_index) + 1
tokenizer.word_index['<SOS>'] = last_idx

In [25]:
temp = [len(each.split()) for each in sents_data]
print('maximum sentence length:', sorted(temp)[-1])

maximum sentence length: 42


### **building the dataset**

In [26]:
def create_shuffles(sent_list, m):
  """
  this function creates m random shuffles of the sentence
  """
  all_combs = []
  for sent in sent_list:
    comb_set = set([])
    for i in range(0, m):
      comb_set.add(text_scramble(sent))
    all_combs.append((sent, comb_set))
  return all_combs

In [27]:
shuffle_pairs = create_shuffles(sents_data, 5)

In [28]:
shuffle_pairs[1][0], shuffle_pairs[1][1]

(' درنخستین دوره مسابقه هاى دوومیدانی داخل سالن آسیا۲۹ تیم حضور داشتند',
 {'آسیا۲۹ دوومیدانی داخل هاى دوره تیم داشتند درنخستین حضور مسابقه سالن',
  'داشتند دوومیدانی هاى آسیا۲۹ تیم حضور دوره سالن مسابقه داخل درنخستین',
  'داشتند مسابقه داخل هاى سالن دوومیدانی آسیا۲۹ درنخستین دوره تیم حضور',
  'هاى آسیا۲۹ داخل دوومیدانی داشتند سالن تیم درنخستین دوره حضور مسابقه',
  'هاى حضور مسابقه درنخستین داشتند تیم داخل دوومیدانی سالن آسیا۲۹ دوره'})

In [29]:
def create_dataframe(pairs):
  """
  creates a dataframe with two columns of original sentence and it's shuffles
  """
  original = []
  shuffled = []
  for pair in pairs:
    for p in pair[1]:
      original.append(pair[0])
      shuffled.append(p)

  df_dict = { 'Shuffled': shuffled, 'Original': original}
  df = pd.DataFrame(df_dict)
  return df


In [30]:
df = create_dataframe(shuffle_pairs)
df = df.sample(frac=1).reset_index(drop=True)
df

,Shuffled,Original
0,غیرمنتظره باشگاه با در فاماگوستا برابر ۱۳ تراب...,باشگاه قبرسی آنارتوسیس فاماگوستا بگونه‌ای غیر...
1,می‌تواند که نیز اتمی استفاده قرار سازی بمب گیر...,غنی سازی اورانیوم روندی است که می‌تواند در تو...
2,توصیه‌ها کیسی شده راه چارچوب موسوم تام صلح وزا...,تام کیسی معاون سخنگوی وزارت امورخارجه آمریکا ...
3,این جاری است گوید عملیات آغازعملیات این عمده د...,ناتومی گوید این عملیات آغازعملیات عمده این ات...
4,کمیسیون نپال پس دولت دوبا ازبرکناری درماه داد ...,گیانندرا پادشاه نپال پس ازبرکناری دولت دوبا د...
...,...,...
173230,نیروهای شدگان علیه بودند آمریکائی با مرتبط توق...,مقامات آمریکائی میگویند توقیف شدگان با حملات ...
173231,شد در یکشنبه دو عراقی کشته ناآرامیهای آمریکائی...,ناآرامیهای روز یکشنبه در عراق باعث کشته شدن د...
173232,ملاقات ژنرال امروز توصیف مبارزه مجدد آمریکا پا...,ژنرال جونز ملاقات امروز را اطمینانی مجدد در پ...
173233,میکند فعلا متزلزل نوار بسی مراعات اسرائیل در ح...,حماس فعلا آتش بسی متزلزل را با اسرائیل در نوا...


In [31]:
def create_dataset(df, train_percent=80):
  data_size = df.shape[0]
  count = math.floor(data_size * (train_percent/100))

  dataset = df['Shuffled'].values
  labels  = df['Original'].values

  train_data   = dataset[:count]
  train_labels = labels[:count]
  test_data   = dataset[count:]
  test_labels = labels[count:]

  return ((train_data, train_labels), (test_data, test_labels))




In [32]:
train_set, test_set = create_dataset(df)
X_train_text, Y_train_text = train_set
X_test_text, Y_test_text = test_set

###**making dataset ready for training**

In [33]:
sos_index = last_idx
def shift_output_sequence(seq_list):
  for seq in seq_list:
    seq.insert(0, sos_index)
  return seq_list


In [34]:
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
Y_train_seq = tokenizer.texts_to_sequences(Y_train_text)
X_test_seq  = tokenizer.texts_to_sequences(X_test_text)
Y_test_seq  = tokenizer.texts_to_sequences(Y_test_text)

In [35]:
MAX_SEQ_LEN = 45  
X_train = keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=MAX_SEQ_LEN)
Y_train = keras.preprocessing.sequence.pad_sequences(Y_train_seq, maxlen=MAX_SEQ_LEN)

X_test  = keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=MAX_SEQ_LEN)
Y_test  = keras.preprocessing.sequence.pad_sequences(Y_test_seq, maxlen=MAX_SEQ_LEN)

X_train_decoder = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(Y_train_seq), maxlen=MAX_SEQ_LEN)
X_test_decoder  = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(Y_test_seq),  maxlen=MAX_SEQ_LEN)

In [36]:
X_train_decoder[:2], Y_train[:2]

(array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0, 33592,
          1139,  8832, 31088, 31089,  6538, 13604,     2,   338,  1139,
         31090,   153,    10,   367,  1487,     3,   513,    42,   357],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0, 33592,   349,   281,   341,  7218,     8,     6,   452,
             2,   224,   106,    45,    44,    38,   210,    43,   558]],
       dtype=int32),
 array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
          1139,  8832, 31088, 31089,  65

In [37]:
X_train[:1],Y_train[:1]

(array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
         13604,  1139,    10,     2, 31089,   338,  1487, 31090,   153,
          8832,   357,    42,   367, 31088,  1139,   513,  6538,     3]],
       dtype=int32),
 array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
          1139,  8832, 31088, 31089,  6538, 13604,     2,   338,  1139,
         31090,   153,    10,   367,  1487,     3,   513,    42,   357]],
       dtype=int32))

In [38]:
VOCAB_SIZE = len(tokenizer.word_index)
VOCAB_SIZE

33592

###**Encoder-Decoder network**

In [39]:
num_samples = X_train.shape[0]

In [40]:
def custom_loss(y_actual,y_pred):
  y_pred = tf.argmax(y_pred, axis=-1, output_type=tf.dtypes.int32)
  find_match =tf.reduce_prod(tf.transpose(y_actual)[...,None]- tf.abs(y_pred[None,...]), 0)
  find_idx = tf.equal(find_match,tf.zeros_like(find_match))
  return (tf.reduce_sum(tf.cast(find_idx, tf.float32)))/MAX_SEQ_LEN


In [48]:
encoder_embedding_size = 64
decoder_embedding_size = 64
lstm_units = 256

np.random.seed(42)
tf.random.set_seed(42)

encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)

encoder_embedding = keras.layers.Embedding(input_dim=VOCAB_SIZE + 1, output_dim=encoder_embedding_size, input_length=MAX_SEQ_LEN, mask_zero=True)(encoder_input)

_, encoder_state_h, encoder_state_c = keras.layers.LSTM(lstm_units, return_state=True)(encoder_embedding)

encoder_state = [encoder_state_h, encoder_state_c]

decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)

decoder_embedding = keras.layers.Embedding(input_dim=VOCAB_SIZE + 2, output_dim=decoder_embedding_size, mask_zero=True)(decoder_input)

decoder_lstm_output = keras.layers.LSTM(lstm_units, return_sequences=True)(decoder_embedding, initial_state=encoder_state)

middle_dense_1 = keras.layers.Dense(lstm_units, activation="relu")(decoder_lstm_output)

decoder_output = keras.layers.Dense(VOCAB_SIZE + 1, activation="softmax")(middle_dense_1)

model = keras.models.Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])

scce = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Nadam(learning_rate=0.005)

model.compile(loss=[scce, custom_loss], loss_weights=[1, 3/num_samples], optimizer=optimizer, metrics=["accuracy"])
model.summary()



Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_8 (Embedding)        (None, None, 64)     2149952     ['input_9[0][0]']                
                                                                                                  
 embedding_9 (Embedding)        (None, None, 64)     2150016     ['input_10[0][0]']               
                                                                                            

In [49]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=10, validation_split=0.15, batch_size=256)

Epoch 1/10
461/461 [==============================] - 299s 589ms/step - loss: 1.5072 - accuracy: 0.5355 - val_loss: 0.3685 - val_accuracy: 0.8936
Epoch 2/10
461/461 [==============================] - 271s 587ms/step - loss: 0.2192 - accuracy: 0.9407 - val_loss: 0.1662 - val_accuracy: 0.9504
Epoch 3/10
461/461 [==============================] - 272s 589ms/step - loss: 0.2772 - accuracy: 0.9318 - val_loss: 0.1014 - val_accuracy: 0.9660
Epoch 4/10
461/461 [==============================] - 269s 584ms/step - loss: 0.0576 - accuracy: 0.9767 - val_loss: 0.0521 - val_accuracy: 0.9780
Epoch 5/10
461/461 [==============================] - 270s 585ms/step - loss: 0.0267 - accuracy: 0.9875 - val_loss: 0.0302 - val_accuracy: 0.9863
Epoch 6/10
461/461 [==============================] - 268s 582ms/step - loss: 0.0498 - accuracy: 0.9808 - val_loss: 0.0272 - val_accuracy: 0.9877
Epoch 7/10
461/461 [==============================] - 269s 584ms/step - loss: 0.0088 - accuracy: 0.9959 - val_loss: 0.0111 -

**almost 99.9% accuracy**

In [61]:
model.save("sentence_model_best.h5")

**model evaluation on test data**

In [ ]:
model = keras.models.load_model("sentence_model_best.h5", custom_objects={"custom_loss":custom_loss})

In [50]:
ids = np.argmax(model.predict([X_test[:1], X_test_decoder[:1]]), axis=-1)
ids, Y_test[:1]

(array([[  428,   428,   428,   428,   428,   428,   428,   428,   428,
           428,   428,   428,   428,   428,   428,   428,   428,     0,
            17,    11,  1024,   359,  2951,    16,     7,     3,  2507,
          1715,  2055,     8,    52,     3,     9,  1182, 16740,  1662,
          1926,   819,    39,    97,   115,     7,     3, 16741,  2404]]),
 array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
            17,    11,  1024,   359,  2951,    16,     7,     3,  2507,
          1715,  2055,     8,    52,     3,     9,  1182, 16740,  1662,
          1926,   819,    39,    97,   115,     7,     3, 16741,  2404]],
       dtype=int32))

In [52]:
print("Evaluate on test data")
results = model.evaluate([X_test, X_test_decoder], Y_test, batch_size=256)
print("test loss, test acc:", results)

Evaluate on test data
136/136 [==============================] - 30s 219ms/step - loss: 0.0091 - accuracy: 0.9970
test loss, test acc: [0.009099681861698627, 0.9969848990440369]


###**Rebuild sentences**

In [53]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

def remove_padding_and_join(texts):
  for i in range(len(texts)):
    no_padding = filter(lambda a: a != '<PAD>', texts[i])
    texts[i] = ' '.join(list(no_padding))
  return texts
    

def reconstruct_predicted_list(texts):
  for i in range(len(texts)):
    pad_start = texts[i].index('<PAD>')
    texts[i] = texts[i][pad_start:]
    no_padding = filter(lambda a: a != '<PAD>', texts[i])
    texts[i] = ' '.join(list(no_padding))
  return texts

def get_predictions_on_test(count):
    Y_preds = model.predict([X_test[:count], X_test_decoder[:count]])
    encoded_argmax  = np.argmax(Y_preds, axis=-1)
    return encoded_argmax, Y_test[:count]

def get_predicted_sentences(predicted):
  predicted_word_list = list(map(sequence_to_text, predicted))
  predicted_sent_list = reconstruct_predicted_list(predicted_word_list)
  return predicted_sent_list

def get_original_sentences(original):
  original_word_list  = list(map(sequence_to_text, original))
  original_sent_list = remove_padding_and_join(original_word_list)
  return original_sent_list 


def rebuild_sentence_from_testset(count=20):
  predicted, original = get_predictions_on_test(count)
  predicted_sent_list = get_predicted_sentences(predicted)
  original_sent_list  = get_original_sentences(original)
  return predicted_sent_list, original_sent_list
  


In [54]:
preds, origs = rebuild_sentence_from_testset()
preds, origs

(['عراق یک هواپیمای بدون خلبان خود را به معرض نمایش گذارده است تا به این وسیله بطلان ادعای کالین پاول وزیر امور خارجه را به ثبوت برساند',
  'یک مقام کره شمالی که درمذاکرات شرکت دارد می\u200cگوید پیونگ یانگ آماده است دربرابر برخورد و رفتار دوستانه آمریکا دست ازبرنامه اتمی خود بردارد',
  'بیش از سی کشور در سطح جهانی استقلال کوسوو را به رسمیت شناخته\u200cاند و این اقدام با اعتراض شدید صربستان مواجه شده است که کوسوو را بخشی از قلمرو خود میداند',
  'سرطان مغزی وی به دنبال حمله مغزی که روزشنبه به وی دست داد کشف گردید',
  'شیرهائی که زمانی در یک باغ وحش خصوصی که عودی پسر صدام حسین برای خود درست کرده بود نگاهداری می\u200cشدند از بغداد به آفریقای جنوبی منتقل می\u200cشوند تا در آنجا زندگی تازه\u200cای را آغاز کنند',
  'علی اصغر سلطانیه گفت هیات کارشناسان آژانس به ریاست هرمن ناکاارتز مذاکرات تکنیکی و تخصصی را با کارشناسان جمهوری اسلامی روز دوشنبه از سر میگیرد',
  'پرزیدنت بوش محاکمه او را منصفانه نامید',
  'مقامات گرجستانی می\u200cگویند افسران روسی روز چهارشنبه در تفلیس پایتخت گرجستان و در باتومی

In [55]:
path = 'data/test_sents.txt'
with open(path, 'r', encoding='utf-8') as f:
  test_sents = [line.strip() for line in f.readlines()]

MAX_LEN = 190
test_sents_cleaned = [text_preprocess(s) for s in test_sents]
len(test_sents_cleaned)

50

In [57]:
test_df = create_dataframe(create_shuffles(test_sents_cleaned, 2))
test_df

,Shuffled,Original
0,اپلیکیشن سفر که کنیم است سفر امروزه باعث شده آ...,اپلیکیشن سفر باعث شده است که امروزه آسوده‌تر س...
1,امروزه اپلیکیشن است باعث که سفر آسوده‌تر کنیم ...,اپلیکیشن سفر باعث شده است که امروزه آسوده‌تر س...
2,و مصادیق به دست در فناوری افراد در است این دنی...,معنی و مصادیق این واژه در حال حاضر در دنیای فن...
3,حال واژه و در در جاه‌طلب مصادیق معنی آینده‌نگر...,معنی و مصادیق این واژه در حال حاضر در دنیای فن...
4,در کودکان وقت خود می‌گذرانند فضای را مجازی بیش...,امروزه کودکان بیشتر وقت خود را در فضای مجازی م...
...,...,...
95,بازداشتی کنید خبرنگاران محکوم را قضائی و رفع مشکل,مشکل قضائی خبرنگاران محکوم و بازداشتی را رفع کنید
96,تازه‌ترین ویدیویی روز و شماست خبرهای و فوری را...,تازه‌ترین خبرهای روز و اخبار فوری به‌صورت ویدی...
97,و و شماست در به‌صورت اخبار رایگان فوری روز دست...,تازه‌ترین خبرهای روز و اخبار فوری به‌صورت ویدی...
98,کسی آنها همه می‌کند که رسانه‌هایی کنترل را می‌...,رسانه‌هایی که همه می‌دانید چه کسی آنها را کنتر...


In [58]:
outer_sents = test_df['Shuffled'].values
label_sents = test_df['Original'].values

outer_seq = tokenizer.texts_to_sequences(outer_sents)
label_seq = tokenizer.texts_to_sequences(label_sents)

X_outer  = keras.preprocessing.sequence.pad_sequences(outer_seq, maxlen=MAX_SEQ_LEN)
X_outer_decoder = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(label_seq), maxlen=MAX_SEQ_LEN)

Y_outer_preds = model.predict([X_outer, X_outer_decoder])
outer_results  = np.argmax(Y_outer_preds, axis=-1)

outer_preds_sents = get_predicted_sentences(outer_results)
result_df = pd.DataFrame({'predictions': outer_preds_sents, 'Original': label_sents})

In [60]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
result_df

,predictions,Original
0,کوکسال سفر باعث شده است که امروزه منطیق سفر کنیم,اپلیکیشن سفر باعث شده است که امروزه آسوده‌تر سفر کنیم
1,کوکسال سفر باعث شده است که امروزه منطیق سفر کنیم,اپلیکیشن سفر باعث شده است که امروزه آسوده‌تر سفر کنیم
2,معنی و ۱۲ام این واژه در حال حاضر در دنیای فناوری به دست افراد ونخستین و راحت‌تر در حال گانه‌ای است,معنی و مصادیق این واژه در حال حاضر در دنیای فناوری به دست افراد جاه‌طلب و آینده‌نگر در حال شکل‌گیری است
3,معنی و ۱۲ام این واژه در حال حاضر در دنیای فناوری به دست افراد ونخستین و راحت‌تر در حال گانه‌ای است,معنی و مصادیق این واژه در حال حاضر در دنیای فناوری به دست افراد جاه‌طلب و آینده‌نگر در حال شکل‌گیری است
4,امروزه کودکان بیشتر وقت خود را در فضای نگزیند شماربزرگی,امروزه کودکان بیشتر وقت خود را در فضای مجازی می‌گذرانند
5,امروزه کودکان بیشتر وقت خود را در فضای نگزیند شماربزرگی,امروزه کودکان بیشتر وقت خود را در فضای مجازی می‌گذرانند
6,استفاده از امضای ایمیل کارها را ۱۲ام می‌کند,استفاده از امضای ایمیل کارها را ساده‌تر می‌کند
7,استفاده از امضای ایمیل کارها را ۱۲ام می‌کند,استفاده از امضای ایمیل کارها را ساده‌تر می‌کند
8,روزهای پایانی سال است و خیلی از ما در شمارمفقود نوشتن اهداف و آرزوهای سال جدید هستیم,روزهای پایانی سال است و خیلی از ما در تکاپوی نوشتن اهداف و آرزوهای سال جدید هستیم
9,روزهای پایانی سال است و خیلی از ما در شمارمفقود نوشتن اهداف و آرزوهای سال جدید هستیم,روزهای پایانی سال است و خیلی از ما در تکاپوی نوشتن اهداف و آرزوهای سال جدید هستیم
